In [ ]:
import pandas as pd
import xml.etree.ElementTree as ET

# Parse the XML file
tree = ET.parse('signal_result_EW.xml')
root = tree.getroot()

# Extract data from XML into a list of dictionaries
data_list = []

for tls in root.findall('tlsState'):
    data_list.append(tls.attrib)

# Convert the list of dictionaries to a dataframe
sumosignalresult = pd.DataFrame(data_list)


# Define the columns to be created
columns = ["SBR", "SBT", "SBL", "WBR", "WBT", "WBL", "NBR", "NBT", "NBL", "EBR", "EBT", "EBL"]

# Define a function to determine the values for the new columns based on conditions
def get_values(row):
    if row['name'] == '2+6':
        if 'y' not in row['state']:
            return ['R', 'R', 'R', 'G', 'G', 'G', 'R', 'R', 'R', 'G', 'G', 'G']
        elif 'y' in row['state']:
            return ['R', 'R', 'R', 'Y', 'Y', 'Y', 'R', 'R', 'R', 'Y', 'Y', 'Y']
        elif row['state'] == 'r'*len(row['state']):
            return ['R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R']
    elif row['name'] == '4+8':
        if 'y' not in row['state']:
            return ['G', 'G', 'G', 'R', 'R', 'R', 'G', 'G', 'G', 'R', 'R', 'R']
        elif 'y' in row['state']:
            return ['Y', 'Y', 'Y', 'R', 'R', 'R', 'Y', 'Y', 'Y', 'R', 'R', 'R']
        elif row['state'] == 'r'*len(row['state']):
            return ['R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R']

# Apply the function and assign the values to the new columns
sumosignalresult[columns] = sumosignalresult.apply(get_values, axis=1, result_type="expand")

condition = sumosignalresult['state'] == 'r'*len(sumosignalresult['state'].iloc[0])
sumosignalresult.loc[condition, columns] = 'R'

sumosignalresult['NBR'] = 'G'
sumosignalresult['SBR'] = 'G'
sumosignalresult['EBR'] = 'G'
sumosignalresult['WBR'] = 'G'

# Remove rows where time is 0
sumosignalresult = sumosignalresult[sumosignalresult['time'] != '0.00']

# Create a new column 'cycle'
sumosignalresult['cycle'] = ((sumosignalresult['time'].astype(float)) / 100).apply(lambda x: int(-(-x // 1)))  # ceil operation

# Melt the dataframe to make it long-form for easier calculation
melted_df = sumosignalresult.melt(id_vars=['cycle'], value_vars=columns, var_name='bound', value_name='state')

# Calculate the occurrence (duration) for each combination of cycle, bound, and state
duration_df = melted_df.groupby(['cycle', 'bound', 'state']).size().reset_index(name='duration')
# Replace state values as specified
state_map = {"G": 1, "R": 0, "Y": 2}
duration_df['state'] = duration_df['state'].replace(state_map)
duration_df = duration_df[['bound', 'cycle', 'state', 'duration']]


# Specifying the desired order for 'bound' column
desired_order = ['SBR', 'SBT', 'SBL', 'WBR', 'WBT', 'WBL', 'NBR', 'NBT', 'NBL', 'EBR', 'EBT', 'EBL']

# Using Categorical to set the order and then sorting
duration_df['bound'] = pd.Categorical(duration_df['bound'], categories=desired_order, ordered=True)

# Sorting by cycle and then by the desired order of 'bound'
duration_df = duration_df.sort_values(by=['cycle', 'bound']).reset_index(drop=True)




# Define custom sorting order for 'bound' and 'state' columns
bound_order = ['SBR', 'SBT', 'SBL', 'WBR', 'WBT', 'WBL', 'NBR', 'NBT', 'NBL', 'EBR', 'EBT', 'EBL']


# Custom sort function for 'state' based on 'bound'
def custom_sort(df):
    if df['bound'].iloc[0] in ['WBR', 'WBT', 'WBL', 'EBR', 'EBT', 'EBL']:
        return df.sort_values(by='state', key=lambda x: x.map({1: 0, 2: 1, 0: 2}))
    else:
        # return df.sort_values(by='state')
        return df.sort_values(by='state', key=lambda x: x.map({1: 0, 2: 1, 0: 2}))

# Apply the custom sort
sorted_duration_df = duration_df.groupby(['cycle', 'bound']).apply(custom_sort).reset_index(drop=True)
sumo_info = sorted_duration_df
sumo_info .to_csv('sumo_info.txt', sep='\t', index=False)








In [ ]:
# Reading the provided txt file and parsing the data
with open("sumo_info.txt", "r") as file:
    lines = file.readlines()

data_dict = {}
for line in lines[1:]:  # Skipping the header
    parts = line.split()
    bound = parts[0]
    cycle = int(parts[1])
    if cycle >=5:
        break
    state = float(parts[2])
    duration = float(parts[3])
    
    if bound not in data_dict:
        data_dict[bound] = [{} for _ in range(cycle, cycle+1)]
    elif len(data_dict[bound]) < cycle:
        data_dict[bound].extend([{}] * (cycle - len(data_dict[bound])))
    
    if state == 0.0:
        data_dict[bound][cycle-1]['red'] = duration
    elif state == 1.0:
        data_dict[bound][cycle-1]['green'] = duration
    elif state == 2.0:
        data_dict[bound][cycle-1]['yellow'] = duration


plot_phase_diagram_hide_yellow_value(data_dict)
